## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key= g_key)

#there's an issue with the API connection
#first test by hand to connect to the g_key, go through the module steps again to make sure the key is working
#go the google api site re-copy in the google key and run again / restart the kernel 

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.41,57.70,80.60,78,40,6.91,light rain
1,1,Khurda,IN,20.18,85.62,82.40,83,20,6.91,haze
2,2,Kushmurun,KZ,52.45,64.63,29.77,94,92,4.00,overcast clouds
3,3,Yatou,CN,37.15,122.38,49.50,84,100,8.30,light rain
4,4,Hobyo,SO,5.35,48.53,82.96,64,94,10.58,overcast clouds


In [46]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [47]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.41,57.70,80.60,78,40,6.91,light rain
1,1,Khurda,IN,20.18,85.62,82.40,83,20,6.91,haze
4,4,Hobyo,SO,5.35,48.53,82.96,64,94,10.58,overcast clouds
5,5,Puerto Baquerizo Moreno,EC,-0.90,-89.60,78.80,83,75,4.61,broken clouds
6,6,Port Blair,IN,11.67,92.75,82.44,75,99,4.27,overcast clouds
7,7,Ribeira Grande,PT,38.52,-28.70,62.60,100,75,3.44,broken clouds
8,8,Taihe,CN,30.87,105.38,54.90,92,100,2.98,overcast clouds
10,10,Aswan,EG,24.09,32.91,74.46,9,0,11.86,clear sky
12,12,Shache,CN,38.42,77.24,60.13,22,3,7.31,clear sky
14,14,Saint-Philippe,RE,-21.36,55.77,80.60,78,75,2.30,broken clouds


In [48]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                467
City                   467
Country                462
Lat                    467
Lng                    467
Max Temp               467
Humidity               467
Cloudiness             467
Wind Speed             467
Current Description    467
dtype: int64

In [49]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [50]:
clean_df = preferred_cities_df.dropna() 


In [51]:
clean_df.count()

City_ID                462
City                   462
Country                462
Lat                    462
Lng                    462
Max Temp               462
Humidity               462
Cloudiness             462
Wind Speed             462
Current Description    462
dtype: int64

In [52]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mahebourg,MU,80.60,light rain,-20.41,57.70,
1,Khurda,IN,82.40,haze,20.18,85.62,
4,Hobyo,SO,82.96,overcast clouds,5.35,48.53,
5,Puerto Baquerizo Moreno,EC,78.80,broken clouds,-0.90,-89.60,
6,Port Blair,IN,82.44,overcast clouds,11.67,92.75,
7,Ribeira Grande,PT,62.60,broken clouds,38.52,-28.70,
8,Taihe,CN,54.90,overcast clouds,30.87,105.38,
10,Aswan,EG,74.46,clear sky,24.09,32.91,
12,Shache,CN,60.13,clear sky,38.42,77.24,
14,Saint-Philippe,RE,80.60,broken clouds,-21.36,55.77,


In [53]:
#test request with a single call
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "-20.41, 57.70"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

test_hotel = hotels["results"][0]["name"]
test_hotel

'Shandrani Beachcomber Resort & Spa'

In [54]:
len(hotels["results"])

20

In [55]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [56]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mahebourg,MU,80.60,light rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
1,Khurda,IN,82.40,haze,20.18,85.62,Bagbaan Villa
4,Hobyo,SO,82.96,overcast clouds,5.35,48.53,Halane Hotel and Restaurent
5,Puerto Baquerizo Moreno,EC,78.80,broken clouds,-0.90,-89.60,Hotel los Algarrobos
6,Port Blair,IN,82.44,overcast clouds,11.67,92.75,Welcomhotel Bay Island Port Blair
7,Ribeira Grande,PT,62.60,broken clouds,38.52,-28.70,Quinta da Meia Eira
8,Taihe,CN,54.90,overcast clouds,30.87,105.38,Meifeng Holiday Inn
10,Aswan,EG,74.46,clear sky,24.09,32.91,Nuba Nile
12,Shache,CN,60.13,clear sky,38.42,77.24,Delong Hotel
14,Saint-Philippe,RE,80.60,broken clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""


In [57]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

#"city: {}, abc: {}".format(n, m)
#f"city: {n}, abc: {m}"
#for index, row in hotel_df.iterrows():
#    print(row)
#    break
    
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

#print(hotel_info[0])
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))